<a href="https://colab.research.google.com/github/alirezaght/iran-bank-card-ocr/blob/main/model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Prepare data

Check the graphic driver

In [2]:
!nvidia-smi

Fri Nov 12 14:11:53 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   57C    P8    31W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## prepare labels

In [3]:
import json
labels = {}
path = 'drive/MyDrive/Colab Notebooks/data'
def prepare_labels():
  notes = open(f'{path}/notes.json')
  categories = json.load(notes)['categories']
  for item in categories:
    labels[item['id']] = item['name']
prepare_labels()
print(labels)



{0: '5', 1: '0', 2: '2', 3: '9', 4: '1', 5: '3', 6: '7', 7: '6', 8: '4', 9: '8', 10: 'exp', 11: 'pan'}


## prepare data

In [17]:
import os
import PIL
import numpy as np
image_width = 300
image_height = 200
pixel_width = 10
pixel_height = 15
x_data = []
y_data = []

def get_category(cat):
  label = labels[cat]
  return [int(label == '0'), int(label == '1'), int(label == '2'), int(label == '3'), int(label == '4'), int(label == '5'), int(label == '6'), int(label == '7'), int(label == '8'), int(label == '9'), int(label == 'pan'), int(label == 'exp')]


def prepare_data():
  list_labels = os.listdir(f'{path}/labels/')
  list_images = {x.split('.')[0]:x for x in os.listdir(f'{path}/images')}  
  for l in list_labels:
    image_name = l.split('.')[0]    
    if image_name in list_images:
      x = np.array(PIL.Image.open(f'{path}/images/{list_images[image_name]}'))      
      x_data.append(x)
      y = np.zeros((image_height // pixel_height, image_width // pixel_width, 17))
      with open(f'{path}/labels/{l}') as file:
        lines = file.read().split('\n')
        for line in lines:
          if line == '':
            continue
          columns = line.split(' ')          
          category = int(columns[0])
          center_x = float(columns[1])
          center_y = float(columns[2])
          width = float(columns[3])
          height = float(columns[4])
          index_x = int(center_x * image_width // pixel_width)
          index_y = int(center_y * image_height // pixel_height)
          result = [1, center_x, center_y, width, height]
          result.extend(get_category(category))
          y[index_y][index_x] = result
        y_data.append(y)

prepare_data()
print(np.array(x_data).shape)
print(np.array(y_data).shape)

(60, 200, 300, 3)
(60, 13, 30, 17)
